<a href="https://colab.research.google.com/github/tj-ie/data_analytics/blob/main/get_years.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_duration.csv')

In [ ]:
df

,start,end
0,2017/1/29,2019/9/10
1,2012/1/30,2015/9/11
2,2019/1/31,NaN
3,2016/2/1,2017/9/13
4,2017/2/2,2019/9/14
5,2017/2/3,NaN
6,2020/2/4,2020/9/16


In [ ]:
df.isnull().any(axis=0)
df.isnull().sum()

# 簡単にmonth列やday列を作れるようになる
# df['month'] = list(pd.Series(df.index).apply(lambda x: x.month))
# df['day'] = list(pd.Series(df.index).apply(lambda x: x.day))

start    0
end      2
dtype: int64

In [ ]:
# 欠損値の補完
# 対象列置換df.replace({'age': {24: 100}})
today = datetime.today().strftime("%Y/%m/%d")
# df.replace({'end': {'NaN': now}})

# fillna
df = df.fillna(today)
df

,start,end
0,2017/1/29,2019/9/10
1,2012/1/30,2015/9/11
2,2019/1/31,2021/05/10
3,2016/2/1,2017/9/13
4,2017/2/2,2019/9/14
5,2017/2/3,2021/05/10
6,2020/2/4,2020/9/16


In [ ]:
# 型変換
df[['start', 'end']] = df[['start', 'end']].astype(str)

oldest_start_date = min(df['start'])
oldest_start_date

'2012/1/30'

In [ ]:
latest_end_date = max(df['end'])
latest_end_date

'2021/05/10'

In [ ]:
oldest_year = int(oldest_start_date[:4])
oldest_year

latest_year = int(latest_end_date[:4])
latest_year

l_duration = []
l_duration.append(oldest_year)

while oldest_year < latest_year:
  oldest_year += 1
  l_duration.append(oldest_year)
  
l_duration

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [ ]:
df[l_duration] = 0
df

,start,end,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,2017/1/29,2019/9/10,0,0,0,0,0,0,0,0,0,0
1,2012/1/30,2015/9/11,0,0,0,0,0,0,0,0,0,0
2,2019/1/31,2021/05/10,0,0,0,0,0,0,0,0,0,0
3,2016/2/1,2017/9/13,0,0,0,0,0,0,0,0,0,0
4,2017/2/2,2019/9/14,0,0,0,0,0,0,0,0,0,0
5,2017/2/3,2021/05/10,0,0,0,0,0,0,0,0,0,0
6,2020/2/4,2020/9/16,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Ensure Types are Correct
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

# Grab Year Columns
year_columns = df.columns[2:]
# Get Year Columns as numpy array as numbers
c = year_columns.to_numpy().astype(int)
# Broadcast start year vs column column (as int) and end year vs year column (as int)
m = ((df['start'].dt.year.to_numpy()[:, None] <= c) &
     (c <= df['end'].dt.year.to_numpy()[:, None]))
# Update year_columns where mask is True
df[year_columns] = df[year_columns].mask(m, 1)
print(df.to_string())


       start        end  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
0 2017-01-29 2019-09-10     0     0     0     0     0     1     1     1     0     0
1 2012-01-30 2015-09-11     1     1     1     1     0     0     0     0     0     0
2 2019-01-31 2021-05-09     0     0     0     0     0     0     0     1     1     1
3 2016-02-01 2017-09-13     0     0     0     0     1     1     0     0     0     0
4 2017-02-02 2019-09-14     0     0     0     0     0     1     1     1     0     0
5 2017-02-03 2021-05-09     0     0     0     0     0     1     1     1     1     1
6 2020-02-04 2020-09-16     0     0     0     0     0     0     0     0     1     0


In [ ]:
# Ensure Types are Correct
df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

for index, row in df.iterrows():
    # Get years from timestamps
    s_year = row['start'].year
    e_year = row['end'].year

    l_years = []
    l_years.append(s_year)

    while s_year < e_year:
        s_year += 1
        l_years.append(s_year)

    for c_year in l_years:
        # Iterate Over Date Columns Only (Exclude Start and End)
        for col_idx, column_name in enumerate(df.columns[2:].values.astype(int)):
            if c_year == column_name:
                # Replace at index, col_idx + 2 because 'start' and 'end'
                # were excluded so col_idx is off by 2
                df.iloc[index, col_idx + 2] = 1
    
print(df.to_string())

       start        end  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
0 2017-01-29 2019-09-10     0     0     0     0     0     1     1     1     0     0
1 2012-01-30 2015-09-11     1     1     1     1     0     0     0     0     0     0
2 2019-01-31 2021-05-10     0     0     0     0     0     0     0     1     1     1
3 2016-02-01 2017-09-13     0     0     0     0     1     1     0     0     0     0
4 2017-02-02 2019-09-14     0     0     0     0     0     1     1     1     0     0
5 2017-02-03 2021-05-10     0     0     0     0     0     1     1     1     1     1
6 2020-02-04 2020-09-16     0     0     0     0     0     0     0     0     1     0


In [ ]:
oldest_year = int(oldest_start_date[:4])

writer = pd.ExcelWriter('/content/drive/MyDrive/Colab Notebooks/data/sample_duration_data.xlsx')
df.to_excel(writer, sheet_name=f'{oldest_year}-{latest_year}', index=False)

for year in l_duration:
  l_df = df.index[df[year] == 1].tolist()
  each_df = df.iloc[l_df]
  
  each_df.to_excel(writer, sheet_name=f'{year}', index=False)

writer.save()
writer.close()